In [ ]:
import pandas as pd
#import seaborn as sbn
import numpy as np
#import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
train_data = pd.read_csv('Task Catagories.csv.csv')

In [ ]:
train_data.shape

In [ ]:
train_data = train_data.drop_duplicates()

In [ ]:
train_data.Category.unique()

In [ ]:
list = train_data.Skill.unique()
print(len(list))

In [ ]:
a = train_data.Skill.value_counts()
skills  = a[a<10].index
filtered_data = train_data[train_data['Skill'].isin(skills)]

train_data = train_data.drop(filtered_data.index)




In [ ]:
train_data['Skill'] = train_data['Skill'].apply(lambda x : x.lower())

In [ ]:
a = ['cloud','devops','deployment']
b = ['ai/ml','data science']
train_data['Category'] = train_data['Category'].apply(lambda x : 'Cloud & Deployment' if x in a else 'DataScience & AI' if x in b else 'database' if x == 'database administration' else x)

In [ ]:
train_data.loc[train_data["Skill"].str.contains("sql"),'Category'] = 'database'

In [ ]:
database = train_data[train_data['Category'] == 'database']
database.Skill.value_counts()

In [ ]:
backend = train_data[train_data['Category'] == 'backend']
backend.Skill.value_counts()

In [ ]:
cloud = train_data[train_data['Category'] == 'cloud']
cloud.Skill.value_counts()

In [ ]:
datascience = train_data[train_data['Category'] == 'data science']
datascience.Skill.value_counts()

In [ ]:
dataadmin = train_data[train_data['Category'] == 'database administration']
dataadmin.Skill.value_counts()

In [ ]:
aiml = train_data[train_data['Category'] == 'ai/ml']
aiml.Skill.value_counts()

In [ ]:
deployment = train_data[train_data['Category'] == 'deployment']
deployment.Skill.value_counts()

In [ ]:
devops = train_data[train_data['Category'] == 'devops']
devops.Skill.value_counts()

In [ ]:
documentation = train_data[train_data['Category'] == 'documentation']
documentation.Skill.value_counts()

In [ ]:
frontend = train_data[train_data['Category'] == 'frontend']
frontend.Skill.value_counts()

In [ ]:
projectManagement = train_data[train_data['Category'] == 'project management']
projectManagement.Skill.value_counts()

In [ ]:
testing = train_data[train_data['Category'] == 'testing']
testing.Skill.value_counts()

In [ ]:
uiux = train_data[train_data['Category'] == 'ui/ux design']
uiux.Skill.value_counts()

In [ ]:
def getSimilarity(detail):
    data = list(train_data['Task Description'])
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([detail] + data)
    cos = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1:])
    return cos[0]
        

In [ ]:
data = list(train_data['Task Description'])
data

In [ ]:
a = getSimilarity("Create a Backend and Generate API")
for i, score in enumerate(a):
    print(f"Query vs Document {i+1}: {score:.4f}")

In [ ]:
def recommendation(detail,category=[],skill=set()):
    similarity = getSimilarity(detail)
    data = train_data.loc[train_data['Category'].isin(category),'Skill']
    required_skill = set(data['Skill'].unique())
    skillPoint = len(list(required_skill & skill))
    

In [ ]:
skills_required_by_category = {
    'Backend' : {'Java','Python','C++'},
    'Frontend' : {'CSS','HTML','React'},
    'Database' : {'MySQL','MongoDB','Redis'},
    'AI & ML' : {'TensorFlow','Pytorch'}
}

In [ ]:
def recommend_users(project_description, project_categories, users, category_skills, 
                     similarity_weight=0.4, skill_weight=0.4, rating_weight=0.2):
    """
    Recommends the best users for a given project.
    
    Parameters:
    - project_description (str): The description of the project.
    - project_categories (list): List of categories related to the project.
    - users (list): A list of user dictionaries containing:
        - 'id': Unique user ID.
        - 'finished_projects': List of { 'description': str, 'category': str }.
        - 'skills': List of skills.
        - 'rating': Numeric rating of the user.
    - category_skills (dict): Mapping of categories to required skills.
    - similarity_weight (float): Weight for project similarity score.
    - skill_weight (float): Weight for skill matching score.
    - rating_weight (float): Weight for user rating.

    Returns:
    - List of top recommended users sorted by their score.
    """
    
    user_scores = []
    vectorizer = TfidfVectorizer()

    # Compute TF-IDF for the given project description
    project_desc_tfidf = vectorizer.fit_transform([project_description])

    for user in users:
        # 1️⃣ **Cosine Similarity Score** (Between Project & User’s Finished Projects)
        user_projects = [p["description"] for p in user["finished_projects"]]
        
        if user_projects:
            user_tfidf = vectorizer.transform(user_projects)
            similarity_scores = cosine_similarity(project_desc_tfidf, user_tfidf)
            max_similarity = np.max(similarity_scores)  # Best match
        else:
            max_similarity = 0  # No finished projects

        # 2️⃣ **Skill Match Score**
        user_skills = set(user["skills"])
        required_skills = set()
        for category in project_categories:
            required_skills.update(category_skills.get(category, []))

        if required_skills:
            skill_score = len(user_skills & required_skills) / len(required_skills)
        else:
            skill_score = 0  # No category-specific skills

        # 3️⃣ **User Rating (Normalized)**
        normalized_rating = user["rating"] / 5.0  # Assuming rating is out of 5

        # 4️⃣ **Final Weighted Score Calculation**
        final_score = (
            (similarity_weight * max_similarity) + 
            (skill_weight * skill_score) + 
            (rating_weight * normalized_rating)
        )

        user_scores.append({"id": user["id"], "score": float(final_score)})

    # Sort Users by Score (Descending)
    user_scores = sorted(user_scores, key=lambda x: x["score"], reverse=True)

    return user_scores[:3]  # Return Top 3 Users

In [ ]:
def recommend_projects(projects, category_skills, user_skills, user_finished_projects, similarity_weight=0.35, skill_weight=0.65, top_n=5):
    """
    Recommend projects based on similarity to user's finished projects and skill match.
    
    Parameters:
    - projects: List of dicts, each with 'description' and 'category'.
    - category_skills: Dict mapping category names to required skills.
    - user_skills: List of user's skills.
    - user_finished_projects: List of dicts with 'description' and 'category'.
    - similarity_weight: Weight for cosine similarity (default 0.6).
    - skill_weight: Weight for skill matching (default 0.4).
    - top_n: Number of top recommendations to return (default 3).
    
    Returns:
    - List of top_n recommended projects.
    """
    
    # ---------------- Step 1: Cosine Similarity Calculation ----------------
    if(not user_finished_projects == []):
        all_descriptions = [proj["Task Description"] for proj in projects] + [proj["Task Description"] for proj in user_finished_projects]
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(all_descriptions)

        project_similarities = []
        for i, project in enumerate(projects):
            similarity_scores = cosine_similarity(tfidf_matrix[i], tfidf_matrix[len(projects):]).flatten()
            project_similarities.append(max(similarity_scores))  # Max similarity with any user project

        # Normalize similarity scores
        max_sim = max(project_similarities) if max(project_similarities) > 0 else 1
        project_similarities = [sim / max_sim for sim in project_similarities]

        all_category = [proj["Category"] for proj in projects] + [proj["Category"] for proj in user_finished_projects]
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(all_category)

        category_similarities = []
        for i, project in enumerate(projects):
            similarity_scores = cosine_similarity(tfidf_matrix[i], tfidf_matrix[len(projects):]).flatten()
            category_similarities.append(max(similarity_scores))  # Max similarity with any user project

        # Normalize similarity scores
        max_sim = max(category_similarities) if max(category_similarities) > 0 else 1
        category_similarities = [sim / max_sim for sim in category_similarities]

        # ---------------- Step 2: Skill Matching Score ----------------
        skill_match_scores = []
        for project in projects:
            category = project["Category"]
            required_skills = category_skills.get(category, [])
            matching_skills = len(set(required_skills) & set(user_skills))
            
            # Normalize skill match score
            skill_match_score = matching_skills / len(required_skills) if required_skills else 0
            skill_match_scores.append(skill_match_score)

        # ---------------- Step 3: Combine Scores and Sort ----------------
        final_scores = [
             similarity_weight*proj_sim +  skill_match * skill_weight + 0.5 * cate
            for proj_sim, skill_match ,cate in zip(project_similarities, skill_match_scores,category_similarities)
        ]

        # Sort projects by final score
        sorted_projects = sorted(zip(projects, final_scores), key=lambda x: x[1], reverse=True)
    else:
        # ---------------- Step 2: Skill Matching Score ----------------
        skill_match_scores = []
        for project in projects:
            category = project["Category"]
            required_skills = category_skills.get(category, [])
            matching_skills = len(set(required_skills) & set(user_skills))
            
            # Normalize skill match score
            skill_match_score = matching_skills / len(required_skills) if required_skills else 0
            skill_match_scores.append(skill_match_score)

        # ---------------- Step 3: Combine Scores and Sort ----------------
        final_scores = [
            skill_match*skill_weight
            for  skill_match in skill_match_scores
        ]

        # Sort projects by final score
        sorted_projects = sorted(zip(projects, final_scores), key=lambda x: x[1], reverse=True)
    # Return top N recommended projects
    return [proj[0] for proj in sorted_projects[:top_n]]

In [ ]:
projects = train_data[['Task Description','Category']].to_dict(orient='records')

In [ ]:
projects = [{'Task Description': 'Implement user authentication', 'Category': 'backend'},
 {'Task Description': 'Create a Button', 'Category': 'frontend'},
 {'Task Description': 'Manage database operations', 'Category': 'database'}]

In [ ]:
category_skills = train_data.groupby('Category')['Skill'].agg(set).to_dict()

In [ ]:
category_skills

In [ ]:
user_finished = [{'Task Description': 'Create a responsive website', 'Category': 'frontend'},

 {'Task Description': 'Develop a mobile-first UI', 'Category': 'frontend'}]
user_skill = ['java','css','figma']

In [ ]:
a = recommend_projects(projects,category_skills,user_skill,user_finished)

In [ ]:
a

In [ ]:
# 🏗️ Sample Project Data
project_description = "Develop a deep learning model for object detection"
project_categories = ["DataScience & AI"]

# 📜 Sample User Data
users = [
    {"id": 6, "finished_projects": [{"description": "Built an AI model for skin detection", "category": "DataScience & AI"}],
     "skills": ["python", "tensorflow"], "rating": 4.8},
    
    {"id": 2, "finished_projects": [{"description": "Developed a machine learning chatbot", "category": "frontend"}],
     "skills": ["python", "css", "figma"], "rating": 4.5},
    
    {"id": 3, "finished_projects": [{"description": "Created a blockchain-based voting system", "category": "backend"}],
     "skills": ["Solidity", "Smart Contracts"], "rating": 4.9}
]
recommended_users = recommend_users(project_description, project_categories, users, category_skills)

print(recommended_users)

In [ ]:
import pandas as pd
import json

# Load CSV data
df = pd.read_csv("recommendation_data.csv")

# Deserialize JSON fields
df["user_skills"] = df["user_skills"].apply(json.loads)
df["user_finished_projects"] = df["user_finished_projects"].apply(json.loads)
df["relevant_projects"] = df["relevant_projects"].apply(json.loads)

In [ ]:
from collections import Counter

def calculate_metrics(recommended_projects, relevant_projects, top_n):
    """
    Calculate Precision@N, Recall@N, and F1-Score@N.
    """
    if top_n == 0:  # Avoid division by zero
        return None, None, None  

    # Get category counts for relevant projects
    relevant_counts = Counter(proj["Category"] for proj in relevant_projects)
    print(relevant_counts)

    # Count true positives by matching categories
    true_positives = 0
    for proj in recommended_projects[:top_n]:
        if relevant_counts[proj["Category"]] > 0:
            true_positives += 1
            relevant_counts[proj["Category"]] -=1
    #true_positives = sum(1 for proj in recommended_projects[:top_n] if relevant_counts[proj["Category"]] > 0)
    print(true_positives)
    # Compute precision and recall
    precision = true_positives / top_n if top_n > 0 else None
    recall = true_positives / len(relevant_projects) if len(relevant_projects) > 0 else None

    # Compute F1-score
    f1_score = (
        2 * (precision * recall) / (precision + recall) 
        if precision is not None and recall is not None and (precision + recall) > 0 
        else None
    )

    return precision, recall, f1_score

#calculate_metrics(recommended_projects,relevant_projects,10)

In [ ]:
unique_dicts = [dict(t) for t in {tuple(d.items()) for d in projects}]

In [ ]:
all_metrics = []

# Iterate over all users
for _, user in df.iterrows():
    # Get user data
    user_skills = user["user_skills"]
    user_finished_projects = user["user_finished_projects"]
    relevant_projects = user["relevant_projects"]
    
    # Run the recommendation algorithm
    recommended_projects = recommend_projects(
        unique_dicts,
        category_skills,
        user_skills,
        user_finished_projects,  
        top_n=10
    )
    
    # Calculate metrics
    precision, recall, f1 = calculate_metrics(
        recommended_projects,
        relevant_projects,
        top_n=10
    )
    
    # Store metrics
    all_metrics.append({
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })

In [ ]:
relevant_projects

In [ ]:
recommended_projects

In [ ]:
# Convert metrics to a DataFrame
metrics_df = pd.DataFrame(all_metrics)

# Calculate mean values
average_precision = metrics_df["precision"].mean()
average_recall = metrics_df["recall"].mean()
average_f1 = metrics_df["f1_score"].mean()

print(f"Average Precision@3: {average_precision:.2f}")
print(f"Average Recall@3: {average_recall:.2f}")
print(f"Average F1-Score@3: {average_f1:.2f}")

In [ ]:
import mysql.connector

# Connect to MySQL
connection = mysql.connector.connect(
    host="localhost",       # Replace with your MySQL host (e.g., "127.0.0.1")
    user="root",   # Your MySQL username
    password="", # Your MySQL password
    database="skillsconnect"  # Your database name
)

In [ ]:
cursor = connection.cursor()

In [ ]:


# Create a cursor object


# Example query: fetch all data from a table
for x in train_data.Skill.unique():
    try:
        # Use parameterized query to prevent SQL injection
        cursor.execute("INSERT INTO skill (name) VALUES (%s)", (x,))
    except Exception as e:
        print(f"Error inserting skill '{x}': {e}")




In [ ]:
for x in train_data.Category.unique():
    try:
        # Use parameterized query to prevent SQL injection
        cursor.execute("INSERT INTO category (name) VALUES (%s)", (x,))
    except Exception as e:
        print(f"Error inserting category '{x}': {e}")

In [ ]:
# Commit the changes to the database
connection.commit()

# Close the connection
cursor.close()
connection.close()

In [ ]:
# Group by 'Category' and count the occurrences of each skill within each category
category_skill_counts = (
    train_data.groupby('Category')['Skill']
    .apply(lambda x: x.str.split(',').explode().str.strip().value_counts())
)

# Display the result
category_skill_counts_df = category_skill_counts.reset_index()
category_skill_counts_df.columns = ['Category', 'Skill', 'Count']
print(category_skill_counts_df)


In [ ]:
train_data['Skill'] = train_data['Skill'].str.split(',')  # Split skills into lists
skills_per_category = (
    train_data.explode('Skill')  # Create separate rows for each skill
    .assign(Skill=lambda df: df['Skill'].str.strip())  # Remove leading/trailing spaces
    .groupby(['Category', 'Skill'])
    .size()
    .reset_index(name='Count')  # Reset index and rename the count column
)

# Visualize the skill counts per category
fig, ax = plt.subplots(figsize=(12, 8))
categories = skills_per_category['Category'].unique()

# Create a bar chart for each category
for category in categories:
    category_data = skills_per_category[skills_per_category['Category'] == category]
    ax.bar(category_data['Skill'], category_data['Count'], label=category)

# Customize the chart
ax.set_title('Skill Counts by Category', fontsize=16)
ax.set_xlabel('Skills', fontsize=14)
ax.set_ylabel('Count', fontsize=14)
ax.tick_params(axis='x', rotation=45)
ax.legend(title='Category', fontsize=10)
plt.tight_layout()

# Show the visualization
plt.show()